In [1]:
import json
import pandas as pd
import numpy as np
from datetime import datetime as dt

from helpers import utils

In [2]:
with open(utils.get_interim_file('flattened.json'), 'r') as fp:
    data = json.load(fp)

In [3]:
df = (
    pd.DataFrame(data.get('properties'))
    .assign(timestamp = lambda x: pd.to_datetime(x.timestamp))
    .assign(firstVisible = lambda x: pd.to_datetime(x.firstVisible))
    .assign(listingUpdated = lambda x: pd.to_datetime(x.listingUpdated))
    .set_index('id')
)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164 entries, 63374392 to 40381836
Data columns (total 19 columns):
addedReduced           164 non-null object
address                164 non-null object
agentName              164 non-null object
buildVersion           164 non-null object
currency               164 non-null object
firstVisible           164 non-null datetime64[ns]
isAuction              164 non-null bool
latitude               164 non-null float64
listingUpdateReason    164 non-null object
listingUpdated         164 non-null datetime64[ns]
longitude              164 non-null float64
numBedrooms            164 non-null int64
numberOfImages         164 non-null int64
price                  164 non-null int64
priceFrequency         164 non-null object
propertySubType        164 non-null object
propertyUrl            164 non-null object
summary                164 non-null object
timestamp              164 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](3), float64(2

In [4]:
df.currency.unique()

array(['GBP'], dtype=object)

In [5]:
df.isAuction.unique()

array([False], dtype=object)

In [6]:
df.numBedrooms.unique()

array([3, 4, 5])

In [7]:
df.propertySubType.unique()

array(['Terraced', 'Flat', 'Semi-Detached', 'Detached', 'House',
       'End of Terrace', 'Not Specified', 'Semi-Detached Bungalow',
       'House Share', 'Detached Bungalow', 'Apartment', 'Cottage',
       'Maisonette', 'Bungalow', 'Town House', 'Private Halls',
       'Ground Flat'], dtype=object)

In [8]:
df.addedReduced.str.split(' ').str.get(0).unique()

array(['Added', 'Reduced'], dtype=object)

In [9]:
df.listingUpdateReason.unique()

array(['new', 'price_reduced'], dtype=object)

In [10]:
df[df.addedReduced.str.find('Added yesterday') == 0].listingUpdateReason.unique()

array(['new'], dtype=object)

In [11]:
# firstVisible can't be trusted, number of instances where it was first visible after listing update
# unless this means first visible in the search results?
df[df.listingUpdated < df.firstVisible].sample(3)

,addedReduced,address,agentName,buildVersion,currency,firstVisible,isAuction,latitude,listingUpdateReason,listingUpdated,longitude,numBedrooms,numberOfImages,price,priceFrequency,propertySubType,propertyUrl,summary,timestamp
id,,,,,,,,,,,,,,,,,,,
63510337,Added on 11/12/2017,"Macfarlane Place, Uphall, Broxburn, EH52","OpenRent, London",4.29.2.1515686502,GBP,2018-01-12 09:04:08,False,55.926520,new,2017-12-11 09:07:15,-3.505468,3,13,695,monthly,Bungalow,/property-to-rent/property-63510337.html,This beautifully presented 3 bedroom property is available to rent immediately. The Landlord has upgraded the property and it is truly offered walk in condition. Downstairs the property comprise... ** Property Reference: 318813 **,2018-01-13 14:55:46
69764795,Added on 30/10/2017,"McAffee Gardens, Armadale, Armadale, EH48","Castlebrae Sales and Letting Ltd, Bathgate Lettings",4.29.2.1515686502,GBP,2017-10-30 17:26:24,False,55.888054,new,2017-10-30 16:00:03,-3.696987,4,21,1050,monthly,Detached,/property-to-rent/property-69764795.html,"Castlebrae - New to market! - Exceptionally well presented large 4 bed detached property, offered FURNISHED, only a few minutes walk from Armadale rail station.",2018-01-13 14:55:46
70884773,Added on 04/01/2018,25 Kirklands Park Street,"Ballantynes, Edinburgh",4.29.2.1515686502,GBP,2018-01-09 01:22:20,False,55.958264,new,2018-01-04 01:21:17,-3.414786,3,6,752,monthly,Terraced,/property-to-rent/property-70884773.html,"A beautifully presented three-bedroom mid terrace Townhouse, located in the sought-after village of Kirkliston.",2018-01-13 14:55:46


In [12]:
df[df.listingUpdateReason == 'price_reduced'].sample(5)

,addedReduced,address,agentName,buildVersion,currency,firstVisible,isAuction,latitude,listingUpdateReason,listingUpdated,longitude,numBedrooms,numberOfImages,price,priceFrequency,propertySubType,propertyUrl,summary,timestamp
id,,,,,,,,,,,,,,,,,,,
70582478,Reduced on 04/01/2018,"Mosside Drive, Blackburn, Bathgate, EH47","Your Move , Bathgate Lettings",4.29.2.1515686502,GBP,2017-12-11 16:40:12,False,55.878082,price_reduced,2018-01-04 12:09:14,-3.622213,3,12,525,monthly,Flat,/property-to-rent/property-70582478.html,"*** AVAILABLE JANUARY *** 3 bedroom Maisonette located in the village of Blackburn. This property has been finished to a great standard and would make a good family home. Property comprises of bright lounge, modern fitted kitchen with new appliances, 2 double bedrooms and a good sized single a...",2018-01-13 14:55:46
50760039,Reduced on 23/11/2017,"Broomhouse Street South, EDINBURGH, Midlothian, EH11","Fineholm , Edinburgh - Lettings",4.29.2.1515686502,GBP,2017-09-27 15:28:44,False,55.924880,price_reduced,2017-11-23 11:35:18,-3.279890,4,9,1150,monthly,House,/property-to-rent/property-50760039.html,"~ HMO property REFURBISHED to excellent standard, 4 bed mid terrace house in popular established residential area of Edinburgh. HMO COMPLIANT!! ~",2018-01-13 14:55:46
70189271,Reduced on 14/12/2017,"151 Jennie Rennies Road, Dunfermline, Fife, KY11","Galbraith, Perth - Lettings",4.29.2.1515686502,GBP,2017-11-16 22:37:54,False,56.060865,price_reduced,2017-12-14 22:38:03,-3.451771,3,10,675,monthly,Not Specified,/property-to-rent/property-70189271.html,"End terraced property on corner plot. Sitting Room, Dining Kitchen, Utility, 3 bedroom, Shower Room and WC. GCH, DG, driveway. Available furnished from 28/1/18. Restrictions: Pets considered. Council Tax: B. Deposit: &pound;1,350. Landord Reg No: 433081/250/21471. EER: D(57).",2018-01-13 14:55:46
68643812,Reduced on 10/10/2017,"78, Mathieson Place, Dunfermline, Fife, KY11","Morgans, Dunfermline",4.29.2.1515686502,GBP,2017-09-01 16:34:32,False,56.062072,price_reduced,2017-10-10 15:33:48,-3.420788,3,10,650,monthly,End of Terrace,/property-to-rent/property-68643812.html,"Furnished or Unfurnished spacious family home in popular residential area affording accommodation over two levels. Ideally placed for all local amenities and schooling. The subjects briefly comprise entrance hall, lounge/dining area, kitchen with washing machine, fridge/freezer, oven and hob. O...",2018-01-13 14:55:46
51749535,Reduced on 13/12/2017,"Buckstone Loan East, Edinburgh, EH10","Home Lettings Scotland, Lasswade - Lettings",4.29.2.1515686502,GBP,2017-11-21 16:52:35,False,55.907280,price_reduced,2017-12-13 16:34:47,-3.198690,3,9,1190,monthly,Semi-Detached,/property-to-rent/property-51749535.html,"If home is the most satisfying place for you to be, then this 3 bed semi detached property would make an ideal rental.",2018-01-13 14:55:46


In [13]:
df.sample(5)

,addedReduced,address,agentName,buildVersion,currency,firstVisible,isAuction,latitude,listingUpdateReason,listingUpdated,longitude,numBedrooms,numberOfImages,price,priceFrequency,propertySubType,propertyUrl,summary,timestamp
id,,,,,,,,,,,,,,,,,,,
69845429,Added on 02/11/2017,"Glencoe, Whitburn, Whitburn, EH47","Castlebrae Sales and Letting Ltd, Bathgate Lettings",4.29.2.1515686502,GBP,2017-11-02 12:02:58,False,55.865660,new,2017-11-02 12:03:03,-3.668700,3,18,695,monthly,Semi-Detached,/property-to-rent/property-69845429.html,"CastleBrae - Delightful 3 bed semi detached property, offered part furnished, in popular location",2018-01-13 14:55:46
52039749,Added on 12/12/2017,"East Pilton Farm Crescent, Pilton, Edinburgh, EH5","Clan Gordon , Edinburgh - Lettings",4.29.2.1515686502,GBP,2017-12-12 20:03:52,False,55.972330,new,2017-12-12 20:03:57,-3.226010,3,11,995,monthly,Flat,/property-to-rent/property-52039749.html,**Available Now** Freshly decorated large and spacious 3 bedroom new build third floor furnished flat (no HMO licence).,2018-01-13 14:55:46
63402316,Added on 04/01/2018,"INVERESK ROAD, MUSSELBURGH, EH21 7BE","D J Alexander, Edinburgh",4.29.2.1515686502,GBP,2018-01-04 17:04:06,False,55.939674,new,2018-01-04 17:04:10,-3.052205,3,14,900,monthly,Flat,/property-to-rent/property-63402316.html,"DEPOSIT &pound;900 - Stylish and beautifully appointed, three bedroom flat within a modern development located in the popular coastal town of Musselburgh, which lies between the city of Edinburgh and East Lothian, AVAILABLE NOW, UNFURNISHED. Musselburgh has an excellent selection of specialist shops, s...",2018-01-13 14:55:46
62215459,Added on 11/10/2017,"Learmonth Crescent, West Calder","Homes 4 U, Bathgate",4.29.2.1515686502,GBP,2017-10-11 16:49:53,False,55.851137,new,2017-10-11 16:49:56,-3.569514,4,8,1000,monthly,Cottage,/property-to-rent/property-62215459.html,"CHARMING VICTORIAN DETACHED COTTAGE located in the heart of West Calder. This spacious, stone built property comprises of vestibule and hallway with beautiful ornate orginal cornacing, high ceilings and cloakroom area. The lounge has traditional solid fuel fire with ornate surround, high ceiling...",2018-01-13 14:55:46
51970008,Added on 07/12/2017,"Learmonth Grove, Comely Bank, City Centre","Murray & Currie, Edinburgh - Lettings",4.29.2.1515686502,GBP,2017-12-07 09:28:15,False,55.957920,new,2017-12-07 09:28:18,-3.221070,3,9,1195,monthly,Flat,/property-to-rent/property-51970008.html,"This three bedroom flat is fully furnished and would suit a professional let. There is no HMO license on the property and not suitable for students.\nThe accommodation has been painted throughout and new carpets added, and comprises of three bedrooms, sitting room, brand new bathroom, modern kitch...",2018-01-13 14:55:46
